## Comparison of calculated vs. provided Angstrom coeffs (440-870) Aeronet V3

In [1]:
import pyaerocom as pya
pya.change_verbosity('critical')

VARS = ['ang4487aer', 'ang4487aer_calc', 'od550aer']
FILTER_NAME = 'WORLD-noMOUNTAINS'
TS_TYPE = 'monthly'
YEAR = 2010

start_str = str(YEAR) 
stop_str = '{}-12-31 23:59:00'.format(YEAR) 

2018-08-29 14:24:05,164:INFO:
Reading aliases ini file: /home/jonasg/github/pyaerocom/pyaerocom/data/aliases.ini
2018-08-29 14:24:06,173:WARNING:
geopy library is not available. Aeolus data read not enabled


### Aeronet V3 Level 2 and Level 1.5

Import data into 2 instance of ``UngriddedData`` class of pyaerocom.

In [2]:
reader = pya.io.ReadUngridded()
reader.logger.setLevel('CRITICAL')

data_l2 = reader.read_dataset(pya.const.AERONET_SUN_V3L2_AOD_DAILY_NAME,
                             vars_to_retrieve=VARS)
data_l15 = reader.read_dataset(pya.const.AERONET_SUN_V3L15_AOD_DAILY_NAME,
                             vars_to_retrieve=VARS)

In [3]:
print(data_l2)
print(data_l15)


Pyaerocom UngriddedData
-----------------------
Contains networks: ['AeronetSunV3Lev2.daily']
Contains variables: ['ang4487aer', 'ang4487aer_calc', 'od550aer']
Contains instruments: []
Total no. of stations: 1165

Pyaerocom UngriddedData
-----------------------
Contains networks: ['AeronetSunV3Lev1.5.daily']
Contains variables: ['ang4487aer', 'ang4487aer_calc', 'od550aer']
Contains instruments: []
Total no. of stations: 1278


Apply altitude filter (i.e. extract all stations that are below 1000 m altitude).

In [ ]:
regfilter = pya.Filter(FILTER_NAME)
data_l2 = regfilter(data_l2)
data_l15 = regfilter(data_l15)

In [ ]:
print(data_l2)
print(data_l15)

Convert all stations to instances of ``StationData`` class (containing time series for all stations). 

In [ ]:
stat_data_l2 = data_l2.to_station_data_all(vars_to_convert=VARS, 
                                           start=start_str,
                                           stop=stop_str,
                                           freq=TS_TYPE)

In [ ]:
stat_data_l15 = data_l15.to_station_data_all(vars_to_convert=VARS, 
                                             start=start_str,
                                             stop=stop_str,
                                             freq=TS_TYPE)

Iterate over all ``StationData`` objects in the two lists and extract data for ``ang4487aer`` (calculated and from files).

In [ ]:
stat_ok_l2 = 0
ang4487aer_file_l2 = []
ang4487aer_calc_l2 = []
for stat in stat_data_l2:
    if stat is not None:
        ang4487aer_file_l2.extend(stat.ang4487aer)
        ang4487aer_calc_l2.extend(stat.ang4487aer_calc)
        stat_ok_l2 += 1

stat_ok_l15 = 0
ang4487aer_file_l15 = []
ang4487aer_calc_l15 = []
for stat in stat_data_l15:
    if stat is not None:
        ang4487aer_file_l15.extend(stat.ang4487aer)
        ang4487aer_calc_l15.extend(stat.ang4487aer_calc)
        stat_ok_l15 += 1

Calculate statistics for scatter plot

In [ ]:
stats_l2 = pya.mathutils.calc_statistics(ang4487aer_calc_l2, ang4487aer_file_l2, lowlim=0)
stats_l15 = pya.mathutils.calc_statistics(ang4487aer_calc_l15, ang4487aer_file_l15, lowlim=0)
print(stats_l2)
print(stats_l15)

In [ ]:
ax = pya.plot.plotscatter_new.plot_scatter(model_vals=ang4487aer_calc_l2,
                                          obs_vals=ang4487aer_file_l2,
                                          model_id='AE 440-870 nm (calculated)', 
                                          obs_id='AE 440-870 nm (from files)',
                                          var_name='ang4487aer',
                                          start=start_str, 
                                          stop=stop_str, 
                                          ts_type=TS_TYPE, 
                                          stations_ok=stat_ok_l2,
                                          filter_name=FILTER_NAME,
                                          lowlim_stats = 0)
ax.set_title('AERONET V3 L2 (2010)')

In [ ]:
ax = pya.plot.plotscatter_new.plot_scatter(model_vals=ang4487aer_calc_l15,
                                           obs_vals=ang4487aer_file_l15,
                                           model_id='AE 440-870 nm (calculated)', 
                                           obs_id='AE 440-870 nm (from files)',
                                           var_name='ang4487aer',
                                           start=start_str, 
                                           stop=stop_str, 
                                           ts_type=TS_TYPE, 
                                           stations_ok=stat_ok_l15,
                                           filter_name=FILTER_NAME,
                                           lowlim_stats = 0)
ax.set_title('AERONET V3 L1.5 (2010)')